In [25]:
import os
import argparse
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
#formatter = argparse.ArgumentDefaultsHelpFormatter
#parser = argparse.ArgumentParser(description='KMEANS Model application', formatter_class=formatter)

#required = parser.add_argument_group(title='required')
#required.add_argument('file', type=str, help='data file to read')
#args = parser.parse_args()
#file_name = args.file
#file_path=  os.path.abspath(file_name)

file_path = "C:\\Users\\Heath\\OneDrive\\Documents\\CS\\CS479\\FinalProject\\cleaned_crime_data_2021_modified_V1.csv"

# read csv into pandas dataframe
data = pd.read_csv(file_path)
data.head()

,DR_NO,DATE.OCC,TIME.OCC,AREA.NAME,Vict.Age,Vict.Sex,Vict.Descent,LAT,LON
0,211607282,4/25/2021 0:00,300,Foothill,32,M,H,34.2293,-118.3675
1,210614453,7/13/2021 0:00,1525,Hollywood,49,M,W,34.1015,-118.3434
2,211115048,11/5/2021 0:00,1700,Northeast,30,M,H,34.0966,-118.2426
3,210811260,3/26/2021 0:00,930,West LA,56,F,O,34.0546,-118.3806
4,210906427,3/1/2021 0:00,1409,Van Nuys,27,F,B,34.1685,-118.4662


In [3]:
#print(data.columns)
data = data.drop(columns=['DR_NO','AREA.NAME'])
data['DATE.OCC'] = data['DATE.OCC'].str.strip(' 0:00')

def convertDateTime(date):
    return datetime.strptime(date, '%m/%d/%Y').timestamp()

data['DATE.OCC'] = data['DATE.OCC'].apply(convertDateTime)
data.head()

,DATE.OCC,TIME.OCC,Vict.Age,Vict.Sex,Vict.Descent,LAT,LON
0,1.619334e+09,300,32,M,H,34.2293,-118.3675
1,1.626160e+09,1525,49,M,W,34.1015,-118.3434
2,1.636096e+09,1700,30,M,H,34.0966,-118.2426
3,1.616742e+09,930,56,F,O,34.0546,-118.3806
4,1.614586e+09,1409,27,F,B,34.1685,-118.4662


In [4]:
#In this current state, not all data is machine readable.
#Vict.Sex and Vict.Descent both need to be converted to a machine readable format.
#We will use one-hot encoding to do this.
#Before we can encode, we will to reduce the number of categories for both Vict.Sex and Vict.Descent
#This is done because a majority of the categories aren't used and will only complicate things later on if they aren't removed
#As seen in our EDA:
    #the 'H' category only occurs 28 times in Vict.Sex, so we will discard it to reduce the dimensions of our one-hot vectors
    #over 90% of the Vict.Descent data lies in 5 descents, the rest we can discard to reduce the dimensions further

#Remove any rows with H in their Vict.Sex column
encode = data.drop(data[data['Vict.Sex']=='H'].index)

#Remove any rows without the specified characters below in their Vict.Descent column
toKeep = ['B','H','O','W','X']
encode = encode.drop(encode[encode['Vict.Descent'].isin(toKeep)==False].index)
encode.head()
#the table printed now only stores records with the most meaningful data

,DATE.OCC,TIME.OCC,Vict.Age,Vict.Sex,Vict.Descent,LAT,LON
0,1.619334e+09,300,32,M,H,34.2293,-118.3675
1,1.626160e+09,1525,49,M,W,34.1015,-118.3434
2,1.636096e+09,1700,30,M,H,34.0966,-118.2426
3,1.616742e+09,930,56,F,O,34.0546,-118.3806
4,1.614586e+09,1409,27,F,B,34.1685,-118.4662


In [5]:
#Now, we will use SKLearns OneHotEncoder to convert this qualitative data into nurmical data
oneHot = OneHotEncoder(categories='auto',handle_unknown='error')

#convert the Vict.Sex column into one-hot encoded vectors
genEncoded = oneHot.fit_transform(encode[['Vict.Sex']]).toarray()
genEncodedlist = list(map(tuple, genEncoded))
genCategories = oneHot.categories_[0]
#need to switch this column header to another title because Vict.Descent also has a header labeled 'X'
genCategories[2] = 'U'

#convert the Vict.Descent column into one-hot encoded vectors
raceEncoded = oneHot.fit_transform(encode[['Vict.Descent']]).toarray()
raceEncodedlist = list(map(tuple, raceEncoded))
raceCategories = oneHot.categories_[0]

#create a dataframe with Vict.Sex and Vict.Descent encoded data
encoded_df = pd.DataFrame(genEncoded, columns=genCategories)
#append the Vict.Descent encoded data to dataframe
encoded_df[raceCategories] = raceEncoded
encoded_df.head(15)

#the data below represent encoded data that a machine can read
#as you can see, the columns with ones represent the category for each row for both Vict.Sex and Vict.Descent

,F,M,U,B,H,O,W,X
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
#same table, but with tupled data
encoded_tuples_df = pd.DataFrame()
encoded_tuples_df[tuple(genCategories)] = genEncodedlist
encoded_tuples_df[tuple(raceCategories)] = raceEncodedlist
encoded_tuples_df.head(15)

,"(F, M, U)","(B, H, O, W, X)"
0,"(0.0, 1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
1,"(0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
2,"(0.0, 1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
4,"(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
5,"(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
6,"(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
7,"(0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
8,"(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
9,"(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"


In [8]:
#we can now discard our old Vict.Sex and Vict.Descent columns
encodedData = data.drop(columns=['Vict.Sex', 'Vict.Descent'])

processed = pd.DataFrame.copy(encodedData)
#append our new encoded columns
processed[encoded_df.columns] = encoded_df
#now we will relabel these column headers to meaningful titles
processed = processed.rename(columns={'F':'Female','M':'Male','U':'Unknown_Sex','B':'Black','H':'Hispanic','O':'Other','W':'White','X':'Unknown_Descent'})
processed.head(20)

,DATE.OCC,TIME.OCC,Vict.Age,LAT,LON,Female,Male,Unknown_Sex,Black,Hispanic,Other,White,Unknown_Descent
0,1.619334e+09,300,32,34.2293,-118.3675,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.626160e+09,1525,49,34.1015,-118.3434,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.636096e+09,1700,30,34.0966,-118.2426,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.616742e+09,930,56,34.0546,-118.3806,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.614586e+09,1409,27,34.1685,-118.4662,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.629529e+09,2351,51,33.9897,-118.2915,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1.631776e+09,1500,24,34.0477,-118.2637,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1.621408e+09,1135,52,34.1505,-118.4622,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,1.616915e+09,1652,57,34.0611,-118.4184,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.614931e+09,700,45,33.9667,-118.2889,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
#same table creation but with the use of tupled data
processed_tuples = pd.DataFrame.copy(encodedData)
processed_tuples[encoded_tuples_df.columns] = encoded_tuples_df
processed_tuples = processed_tuples.rename(columns={('F','M','U'): 'Vict.Sex (F,M,U)', ('B','H','O','W','X'):'Vict.Descent (B,H,O,W,X)'})
processed_tuples.head(20)

,DATE.OCC,TIME.OCC,Vict.Age,LAT,LON,"Vict.Sex (F,M,U)","Vict.Descent (B,H,O,W,X)"
0,1.619334e+09,300,32,34.2293,-118.3675,"(0.0, 1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
1,1.626160e+09,1525,49,34.1015,-118.3434,"(0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
2,1.636096e+09,1700,30,34.0966,-118.2426,"(0.0, 1.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,1.616742e+09,930,56,34.0546,-118.3806,"(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
4,1.614586e+09,1409,27,34.1685,-118.4662,"(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
5,1.629529e+09,2351,51,33.9897,-118.2915,"(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
6,1.631776e+09,1500,24,34.0477,-118.2637,"(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
7,1.621408e+09,1135,52,34.1505,-118.4622,"(0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)"
8,1.616915e+09,1652,57,34.0611,-118.4184,"(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
9,1.614931e+09,700,45,33.9667,-118.2889,"(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
